In [0]:
#import spark.implicits._
#import org.apache.spark.sql.functions._
from pyspark.sql.functions import col

from scipy import stats as st
import numpy as np
import pandas as pd
import math as mt
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns


from IPython.display import display
from IPython.display import HTML


In [0]:
# Configurar el storage y las credenciales
storage_account_name = "etl2"
container_name = "container1"
sas_token = "sp=racwdlmeop&st=2024-10-04T19:56:30Z&se=2024-11-05T03:56:30Z&sv=2022-11-02&sr=c&sig=fInMPOlOvq8bb4UA0lIibhUNflYplQ3Q9mZK8BKyfKY%3D"


In [0]:
# Montar el Data Lake Gen2
dbutils.fs.updateMount(  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
  mount_point = f"/mnt/{container_name}",
  extra_configs = {f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token}
)

True

In [0]:
spark.conf.set(
  f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
  f"{sas_token}"
)

In [0]:
display(dbutils.fs.ls("/mnt/container1"))

[FileInfo(path='dbfs:/mnt/container1/cabs.csv', name='cabs.csv', size=31998, modificationTime=1728164694000),
 FileInfo(path='dbfs:/mnt/container1/datos.csv', name='datos.csv', size=28955, modificationTime=1728081187000),
 FileInfo(path='dbfs:/mnt/container1/neighborhoods.csv', name='neighborhoods.csv', size=1521, modificationTime=1728164694000),
 FileInfo(path='dbfs:/mnt/container1/new_retail_data.csv', name='new_retail_data.csv', size=84920745, modificationTime=1728071170000),
 FileInfo(path='dbfs:/mnt/container1/trips.csv', name='trips.csv', size=10817, modificationTime=1728164694000),
 FileInfo(path='dbfs:/mnt/container1/weather_records.csv', name='weather_records.csv', size=8187, modificationTime=1728164694000)]

In [0]:
print("Hola Mundo!")

Hola Mundo!


In [0]:
# Cargar el dataset desde el Data Lake Gen2
cabs = spark.read.csv("/mnt/container1/cabs.csv", header=True, inferSchema=True)
trips = spark.read.csv("/mnt/container1/trips.csv", header=True, inferSchema=True)
neighborhoods = spark.read.csv("/mnt/container1/neighborhoods.csv", header=True, inferSchema=True)
weather_records = spark.read.csv("/mnt/container1/weather_records.csv", header=True, inferSchema=True)
datos = spark.read.csv("/mnt/container1/datos.csv", header=True, inferSchema=True)



In [0]:
cabs.cache()
trips.cache()
neighborhoods.cache()
weather_records.cache()
datos.cache()



DataFrame[Date and time: timestamp, Temperature: double, Description: string]

In [0]:
cabs = cabs.dropDuplicates()
trips = trips.dropDuplicates()
neighborhoods = neighborhoods.dropDuplicates()
weather_records = weather_records.dropDuplicates()
datos = datos.dropDuplicates()


In [0]:
cabs.createOrReplaceTempView("cabs")
trips.createOrReplaceTempView("trips")
neighborhoods.createOrReplaceTempView("neighborhoods")
weather_records.createOrReplaceTempView("weather_records")
datos.createOrReplaceTempView("datos")


In [0]:
filtro1 = spark.sql(""" 
        SELECT company_name, COUNT(trips.trip_id) AS trips_amount
        FROM cabs INNER JOIN trips ON cabs.cab_id = trips.cab_id
        WHERE DATE(start_ts) IN ('2017-11-15', '2017-11-16')
        GROUP BY cabs.company_name
        ORDER BY trips_amount DESC
""")


In [0]:
trips.show()

+-------+------+----------------+----------------+----------------+--------------+------------------+-------------------+
|trip_id|cab_id|        start_ts|          end_ts|duration_seconds|distance_miles|pickup_location_id|dropoff_location_id|
+-------+------+----------------+----------------+----------------+--------------+------------------+-------------------+
|     19|  3953|03/11/2017 19:00|03/11/2017 19:00|             453|           1.3|                 0|                  0|
|      3|  1768|03/11/2017 14:00|03/11/2017 15:00|             318|           0.9|                 0|                  0|
|     14|  1943|01/11/2017 20:00|01/11/2017 20:00|             120|           0.5|                 0|                  0|
|     10|  3981|28/11/2017 00:00|28/11/2017 00:00|             136|           0.8|                 0|                  0|
|     15|   266|06/11/2017 11:00|06/11/2017 11:00|             180|           0.5|                 0|                  0|
|      4|  4274|24/11/20

In [0]:
filtro2 = spark.sql("""
        select *
        from 
            (select company_name, count(trip_id) as trips_amount 
            from cabs inner join trips on cabs.cab_id=trips.cab_id
        where company_name like '%Yellow%'  and start_ts::date between '2017-11-01' and '2017-11-07'
            group by cabs.company_name) as sub1 union
            (select company_name, count(trip_id) as trips_amount
        from cabs inner join trips on cabs.cab_id=trips.cab_id
        where company_name like '%Blue%'  and start_ts::date between '2017-11-01' and '2017-11-07'
            group by cabs.company_name) 
""")

In [0]:
filtro3 = spark.sql("""
                    select
                        CASE
                        WHEN company_name ='Flash Cab' THEN  company_name
                        WHEN company_name = 'Taxi Affiliation Services' THEN  company_name
                        ELSE 'Other' END AS company,
                        count(trips.trip_id) as trips_amount
                        
                    FROM
                        cabs inner join trips on cabs.cab_id=trips.cab_id
                    WHERE 
                        trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'
                    group by company
                    ORDER BY 
                        trips_amount DESC;
                    """)

In [0]:
filtro4= spark.sql("""
                select neighborhood_id, name from neighborhoods 
                where name in ('O''Hare','Loop');
        """)

In [0]:
filtro5=spark.sql("""
                  select ts, 
                    case WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN  'Bad'
                    else 'Good' end as weather_conditions
                from weather_records
""")


In [0]:
filtro6=spark.sql("""
                  select start_ts, 
                    case WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN  'Bad'
                    else 'Good' end as weather_conditions,
                    duration_seconds
                from trips join weather_records on trips.start_ts=weather_records.ts
                where pickup_location_id=50 and dropoff_location_id=63 and EXTRACT (DOW from trips.start_ts) = 6
                order by trip_id
                """)


In [0]:
filtro6.show()

+--------+------------------+----------------+
|start_ts|weather_conditions|duration_seconds|
+--------+------------------+----------------+
+--------+------------------+----------------+



In [0]:
filtro1.write.mode("overwrite").parquet("/mnt/container1/sql_result_01")
filtro2.write.mode("overwrite").parquet("/mnt/container1/sql_result_02")
filtro3.write.mode("overwrite").parquet("/mnt/container1/sql_result_03")
filtro4.write.mode("overwrite").parquet("/mnt/container1/sql_result_04")
filtro5.write.mode("overwrite").parquet("/mnt/container1/sql_result_05")
filtro6.write.mode("overwrite").parquet("/mnt/container1/sql_result_06")
